# zarr-ification of CESM-LE for AWS

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import shutil

import yaml

import xarray as xr

import intake
from distributed.utils import format_bytes
from operator import mul
from functools import reduce

## Spinup cluster

In [2]:
from ncar_jobqueue import NCARCluster
cluster = NCARCluster(memory="50GB")
cluster

In [15]:
cluster.adapt(minimum=0, maximum=40, wait_count=60)

In [4]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://10.12.205.200:33098 Dashboard: https://jupyterhub.ucar.edu/ch/user/abanihi/proxy/33876/status,Cluster Workers: 30 Cores: 30 Memory: 1.50 TB


## Connect to intake-esm database

In [5]:
col = intake.open_esm_metadatastore(collection_name='CESM1-LE')
col.df.head()

,resource,resource_type,direct_access,experiment,case,component,stream,variable,date_range,member_id,file_fullpath,file_basename,file_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
0,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,18000101-18991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.180001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
1,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,08000101-08991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.080001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
2,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,04020101-04991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.040201...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
3,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,10000101-10991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.100001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
4,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,11000101-11991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.110001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN


## Helper functions

In [6]:
def collection_summary(col):
    df = col.df
    print("There are:\n")
    print(f"\t1) {df.experiment.nunique()} experiments: \n\n\t\t{df.experiment.unique()}\n")
    print(f"\t2) {df.case.nunique()} cases: \n\n\t\t{df.case.unique()[:5]} etc...\n")
    print(f"\t3) {df.component.nunique()} components: \n\n\t\t{df.component.unique()}\n")
    print(f"\t4) {df.stream.nunique()} streams: \n\n\t\t{df.stream.unique()}\n")
    print(f"\t5) {df.variable.nunique()} variables: \n\n\t\t{df.variable.unique()[:10]} etc...\n")
    

In [7]:
def print_query_info(query_results):
    variables = sorted(query_results.variable.unique().tolist())
    files = sorted(query_results.file_fullpath.unique().tolist())
    date_ranges = sorted(query_results.date_range.unique())
    cases = sorted(query_results.case.unique())
    members = sorted(query_results.member_id.unique())
    print(f"This query: {query} returned:\n\n- {len(variables)} unique variable(s)\n\n")
    print(f"- Number of files: {len(files)}\n\n")
    #print(f"- {len(members)} member(s): {members}\n\n")
    #print(f"Date ranges: {date_ranges}\n\n")
    #print(f"- {len(cases)} case(s)\n\n")
    

In [8]:
def zarr_store(exp, cmp, frequency, var, write=False, dirout='/glade/scratch/abanihi/lens-aws'):
    """ Create zarr store name/path """
    path = f'{dirout}/cesmLE-{exp}-{cmp}-{frequency}-{var}.zarr'    
    if write and os.path.exists(path):
        shutil.rmtree(path)
    print(path)
    return path     

In [9]:
def read_data(cat, xr_open, member_chunks):
    dd = cat.to_xarray(**xr_open)
    _, ds = dd.popitem()
    return ds.chunk(member_chunks)

def print_ds_info(ds, var):
    dt = ds[var].dtype
    itemsize = dt.itemsize
    chunk_size = ds[var].data.chunksize
    size = format_bytes(ds.nbytes)
    _bytes = reduce(mul, chunk_size) * itemsize
    chunk_size_bytes = format_bytes(_bytes)
    
    print(f"Variable name: {var}")
    print(f"Chunk shape: {chunk_size}")
    print(f"Dataset shape: {ds[var].shape}")
    print(f"Chunk size: {chunk_size_bytes}")
    print(f"Dataset size: {size}")
    #print(f"Chunks: {ds[var].chunks}")

In [10]:
collection_summary(col)

There are:

	1) 6 experiments: 

		['CTRL' 'CTRL-AMIP' 'CTRL-SLAB-OCN' '20C' '20C-OIC' 'RCP85']

	2) 93 cases: 

		['b.e11.B1850C5CN.f09_g16.005' 'f.e11.F1850C5CN.f09_f09.001'
 'e.e11.E1850C5CN.f09_g16.001' 'b.e11.B20TRC5CNBDRD.f09_g16.105'
 'b.e11.B20TRC5CNBDRD.f09_g16.104'] etc...

	3) 5 components: 

		['atm' 'ocn' 'lnd' 'rof' 'ice']

	4) 10 streams: 

		['cam.h1' 'cam.h0' 'pop.h.nday1' 'pop.h' 'clm2.h1' 'clm2.h0' 'rtm.h1'
 'rtm.h0' 'cice.h1' 'cice.h']

	5) 985 variables: 

		['T200' 'Q200' 'FLNSC' 'Q500' 'PRECL' 'QBOT' 'TREFHTMN' 'FSNTOA' 'TAUX'
 'TAUY'] etc...



## Load variable/chunking specs

In [11]:
with open("../variables_chunking_spec.yaml") as f:
    specs = yaml.safe_load(f)
specs

{'ocn': {'pop.h': {'variable_category': {'2D': {'variables': ['SST', 'SSH'],
     'chunks': {'time': 12, 'member_id': 40}},
    '3D': {'variables': ['SALT'],
     'chunks': {'time': 12, 'z_t': 1, 'member_id': 40}}},
   'frequency': 'monthly'}},
 'ice': {'cice.h': {'variable_category': {'NH': {'variables': ['aice_nh',
      'hi_nh'],
     'chunks': {'time': 12, 'member_id': 40}},
    'SH': {'variables': ['aice_sh', 'hi_sh'],
     'chunks': {'time': 12, 'member_id': 40}}},
   'frequency': 'monthly'}},
 'atm': {'cam.h0': {'variable_category': {'3D': {'variables': ['T',
      'U',
      'V',
      'Q',
      'Z3'],
     'chunks': {'time': 12, 'lev': 1, 'member_id': 40}},
    '2D': {'variables': ['FLNS',
      'FLNSC',
      'FLUT',
      'FSNS',
      'FSNSC',
      'FSNTOA',
      'ICEFRAC',
      'LHFLX',
      'PRECC',
      'PRECL',
      'PRECSC',
      'PRECSL',
      'PSL',
      'SHFLX',
      'TMQ',
      'TREFHT',
      'TREFHTMN',
      'TREFHTMX',
      'TS'],
     'chunks': {'

## Process `land` component (realm)

In [12]:
component_list = ['lnd']

experiment_list = ['20C', 'RCP85'][1:]
experiment_list

['RCP85']

In [13]:
def process_data(experiment_list, component_list, specs):
    for exp in experiment_list:
        print("="*100)
        for cmp in component_list:
            specs_ = specs.copy()
            for stm, stm_specs in specs_[cmp].items():
                frequency = stm_specs['frequency']
                var_cats = stm_specs['variable_category']
                for var_cat, var_cat_values in var_cats.items():
                    variables = var_cat_values['variables']
                    chunks = var_cat_values['chunks'].copy()
                    member_chunks = {}
                    member_chunks['member_id'] = chunks.pop('member_id', 40)
                    xr_open = dict(chunks=chunks, decode_times=True, decode_coords=False)
                    for variable in variables:
                        query = dict(experiment=exp, component=cmp, stream=stm, variable=variable)
                        cat = col.search(**query)
                        print("#"*100)
                        print(f'Query= {query}')
                        results = cat.query_results
                        if not results.empty:
                            #print("*"*100)
                            #print_query_info(results)
                            try:
                                ds = read_data(cat, xr_open, member_chunks)
                                if exp == '20C':
                                    ds = ds.sel(time=slice("1920-02", None))
                                print("+"*35)
                                if variable in {'aice_nh', 'aice_sh'}:
                                    variable = 'aice'
                                elif variable in {'hi_nh', 'hi_sh'}:
                                    variable = 'hi'
                                print_ds_info(ds, variable)
                                print("+"*35)
                                print(ds)
                                store = zarr_store(exp, cmp, frequency, variable, write=True)
                                ds.to_zarr(store, consolidated=True)
                                del ds
                            except Exception as e:
                                raise(e)
                        else:
                            print(f"No results found for query={query}")
                            continue


In [14]:
process_data(experiment_list, component_list, specs)

####################################################################################################
Query= {'experiment': 'RCP85', 'component': 'lnd', 'stream': 'clm2.h0', 'variable': 'FSNO'}



+++++++++++++++++++++++++++++++++++
Variable name: FSNO
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
    DZSOI      


+++++++++++++++++++++++++++++++++++
Variable name: H2OSNO
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
    DZSOI    


+++++++++++++++++++++++++++++++++++
Variable name: QRUNOFF
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
    DZSOI   


+++++++++++++++++++++++++++++++++++
Variable name: RAIN
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
    DZSOI      


+++++++++++++++++++++++++++++++++++
Variable name: SNOW
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
    DZSOI      


+++++++++++++++++++++++++++++++++++
Variable name: SOILWATER_10CM
Chunk shape: (40, 12, 192, 288)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.11 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:         (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat             (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd         (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak          (levlak) float32 0.05 0.6 2.1 4.6 ... 25.6 34.325 44.775
  * lon             (lon) float32 0.0 1.25 2.5 3.75 ... 356.25 357.5 358.75
  * time            (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id       (member_id) int64 1 2 3 4 5 6 7 ... 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW             (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(15, 192, 288)>
   


+++++++++++++++++++++++++++++++++++
Variable name: SOILLIQ
Chunk shape: (40, 12, 1, 192, 288)
Dataset shape: (40, 1140, 15, 192, 288)
Chunk size: 106.17 MB
Dataset size: 151.31 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (levgrnd, lat, lon) float32 dask.array<shape=(15, 192, 288), chunksize=(1, 192, 288)>
    D

distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
